In [ ]:
#DONE 1. demonstrate test locally
    # bonus: set up tensorboard
# 2. demonstrate test at 
# 3. change to smaller ResNet
# 4. demonstrate on a dcm slice
# 5. demonstrate in 3d

In [9]:
import importlib

In [ ]:
importlib.reload()

In [ ]:
keras.layers.convolutional.Conv2D()

In [1]:
import keras

Using TensorFlow backend.


In [1]:
import cv2
import os
import keras
import train_frcnn
import importlib

Using TensorFlow backend.


In [21]:
importlib.reload(train_frcnn)

<module 'train_frcnn' from '/Users/clintonwang/Documents/Work/voi-detection/train_frcnn.py'>

In [23]:
os.system("python train_frcnn.py")

256

In [27]:
from functools import partial

In [41]:
basetwo = partial(print, sep='/', end='.')

In [42]:
basetwo('hi','there','mate')

hi/there/mate.

In [18]:
x_dim = 50
y_dim = 50
z_dim = 3
dim_tuple = (z_dim, x_dim, y_dim)

train_frcnn
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Generate dummy data
train_size = 10
import numpy as np
data = np.random.random((train_size, z_dim, x_dim, y_dim))
labels = np.random.randint(2, size=(train_size, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=2)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
10/10 [==============================] - 11s - loss: 1.7511 - acc: 0.2000
Epoch 2/10
10/10 [==============================] - 1s - loss: 0.7666 - acc: 1.0000
Epoch 3/10
10/10 [==============================] - 1s - loss: 0.7474 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 1s - loss: 0.7412 - acc: 1.0000
Epoch 5/10
10/10 [==============================] - 1s - loss: 0.7357 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 1s - loss: 0.7305 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 1s - loss: 0.7254 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 1s - loss: 0.7205 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 1s - loss: 0.7156 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 1s - loss: 0.7108 - acc: 1.0000
